In [1]:
import pandas as pd

In [3]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
dim_movies = movies_df.copy()
dim_movies.rename(columns={'movieId': 'MovieID', 'title': 'Title', 'genres': 'Genres'}, inplace=True)
dim_movies.insert(0, 'MovieKey', range(1, 1 + len(dim_movies)))
dim_movies.head()

,MovieKey,MovieID,Title,Genres
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,3,Grumpier Old Men (1995),Comedy|Romance
3,4,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,5,Father of the Bride Part II (1995),Comedy


In [8]:
unique_users = ratings_df['userId'].unique()
dim_users = pd.DataFrame(unique_users, columns=['UserID'])
dim_users.insert(0, 'UserKey', range(1, 1 + len(dim_users)))
dim_users.head()

,UserKey,UserID
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [9]:
ratings_df['rating_date'] = pd.to_datetime(ratings_df['timestamp'], unit='s')

In [10]:
unique_dates = ratings_df['rating_date'].dt.date.unique()
dim_date = pd.DataFrame(unique_dates, columns=['FullDate'])

In [11]:
dim_date['DateKey'] = dim_date['FullDate'].apply(lambda x: int(x.strftime('%Y%m%d')))
dim_date.head()

,FullDate,DateKey
0,2000-07-30,20000730
1,2000-08-08,20000808
2,2015-10-24,20151024
3,2011-05-27,20110527
4,2001-04-10,20010410


In [ ]:
dim_date['FullDate'] = pd.to_datetime(dim_date['FullDate'])
dim_date['Year'] = dim_date['FullDate'].dt.year
dim_date['Quarter'] = dim_date['FullDate'].dt.quarter
dim_date['Month'] = dim_date['FullDate'].dt.month
dim_date['Day'] = dim_date['FullDate'].dt.day
dim_date['DayOfWeek'] = dim_date['FullDate'].dt.day_name()

In [13]:
dim_date = dim_date[['DateKey', 'FullDate', 'Year', 'Quarter', 'Month', 'Day', 'DayOfWeek']]
dim_date.head()

,DateKey,FullDate,Year,Quarter,Month,Day,DayOfWeek
0,20000730,2000-07-30,2000,3,7,30,Sunday
1,20000808,2000-08-08,2000,3,8,8,Tuesday
2,20151024,2015-10-24,2015,4,10,24,Saturday
3,20110527,2011-05-27,2011,2,5,27,Friday
4,20010410,2001-04-10,2001,2,4,10,Tuesday


In [18]:
fact_ratings = ratings_df.copy()
fact_ratings.head()

,userId,movieId,rating,timestamp,rating_date
0,1,1,4.0,964982703,2000-07-30 18:45:03
1,1,3,4.0,964981247,2000-07-30 18:20:47
2,1,6,4.0,964982224,2000-07-30 18:37:04
3,1,47,5.0,964983815,2000-07-30 19:03:35
4,1,50,5.0,964982931,2000-07-30 18:48:51


In [19]:
fact_ratings['DateKey'] = fact_ratings['rating_date'].dt.strftime('%Y%m%d').astype(int)

In [20]:
fact_ratings = pd.merge(fact_ratings, dim_movies[['MovieID', 'MovieKey']], left_on='movieId', right_on='MovieID')
fact_ratings = pd.merge(fact_ratings, dim_users[['UserID', 'UserKey']], left_on='userId', right_on='UserID')
fact_ratings.head()

,userId,movieId,rating,timestamp,rating_date,DateKey,MovieID,MovieKey,UserID,UserKey
0,1,1,4.0,964982703,2000-07-30 18:45:03,20000730,1,1,1,1
1,1,3,4.0,964981247,2000-07-30 18:20:47,20000730,3,3,1,1
2,1,6,4.0,964982224,2000-07-30 18:37:04,20000730,6,6,1,1
3,1,47,5.0,964983815,2000-07-30 19:03:35,20000730,47,44,1,1
4,1,50,5.0,964982931,2000-07-30 18:48:51,20000730,50,47,1,1


In [21]:
fact_ratings = fact_ratings[['rating', 'MovieKey', 'UserKey', 'DateKey']]
fact_ratings.rename(columns={'rating': 'Rating'}, inplace=True)
fact_ratings.insert(0, 'RatingID', range(1, 1 + len(fact_ratings)))
fact_ratings.head()

,RatingID,Rating,MovieKey,UserKey,DateKey
0,1,4.0,1,1,20000730
1,2,4.0,3,1,20000730
2,3,4.0,6,1,20000730
3,4,5.0,44,1,20000730
4,5,5.0,47,1,20000730


In [22]:
dim_movies.to_csv('dim_movies.csv', index=False)
dim_users.to_csv('dim_users.csv', index=False)
dim_date.to_csv('dim_date.csv', index=False)
fact_ratings.to_csv('fact_ratings.csv', index=False)